# RAG Baseline (XML)

Dieses Notebook baut eine einfache RAG-Pipeline auf Basis der XML-Datei `data/grundschutz.xml`.


## 0) Voraussetzungen
- `.env` in `notebooks/` anlegen (Kopie von `.env.example`)
- Qdrant lokal mit Docker starten (Standard: `http://localhost:6333`)

```bash
docker pull qdrant/qdrant
docker run -d --name qdrant \
  -p 6333:6333 -p 6334:6334 \
  -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
  qdrant/qdrant
```

- `uv sync` oder `pip install ...` mit `litellm`, `qdrant-client`, `python-dotenv`


In [54]:
from pathlib import Path
import sys

import xml.etree.ElementTree as ET

# Make notebooks/ importable when running from Jupyter
NOTEBOOK_DIR = Path.cwd()
if (NOTEBOOK_DIR / "litellm_client.py").exists():
    sys.path.insert(0, str(NOTEBOOK_DIR))
elif (NOTEBOOK_DIR / "notebooks" / "litellm_client.py").exists():
    sys.path.insert(0, str(NOTEBOOK_DIR / "notebooks"))

from litellm_client import (
    chat_completion,
    get_embeddings,
    get_qdrant_client,
    load_llm_config,
    load_vectordb_config,
)

DATA_PATH = Path("../data/grundschutz.xml")


## 1) Vektor-Datenbank initialisieren
Wir legen (falls nötig) die Collection an und prüfen die Verbindung.


In [55]:
# Qdrant-Verbindung prüfen
from qdrant_client.http import models as qmodels

llm_cfg = load_llm_config()
vec_cfg = load_vectordb_config()
client = get_qdrant_client(vec_cfg)

collection_name = vec_cfg.collection or "grundschutz_xml"


/Users/felixboelter/Documents/GitHub/pilotprojekt-GrundschutzKI/notebooks/litellm_client.py:177: UserWarning: Api key is used with an insecure connection.
  return QdrantClient(url=url, api_key=cfg.api_key)


## 2) XML laden und in Text-Chunks aufteilen
Wir extrahieren Text aus der XML, säubern leicht und erstellen Chunks.


In [ ]:
# XML laden (robust gegen BOM/Encoding/HTML-Header)
from io import BytesIO
from lxml import etree as LET

raw = DATA_PATH.read_bytes()

# Entferne UTF-8 BOM falls vorhanden
if raw.startswith(b"\xef\xbb\xbf"):
    raw = raw[3:]

# Heuristisch zu Text dekodieren
try:
    text = raw.decode("utf-8")
except UnicodeDecodeError:
    text = raw.decode("latin-1", errors="ignore")

# Falls ein nicht-XML-Header davor steht, skippe bis zum ersten '<'
lt = text.find("<")
if lt > 0:
    text = text[lt:]

if not text.lstrip().startswith("<"):
    preview = text[:200].replace("\n", " ")
    raise ValueError(f"Datei sieht nicht wie XML aus. Vorschau: {preview}")

# LXML mit recover=True
parser = LET.XMLParser(recover=True, encoding="utf-8")
try:
    root = LET.fromstring(text.encode("utf-8"), parser=parser)
except LET.XMLSyntaxError as exc:
    raise ValueError("XML konnte nicht geparst werden. Bitte Dateiformat pruefen.") from exc

# Heuristik: alle Textknoten sammeln
texts = [t for t in root.itertext() if t and t.strip()]

len(texts)


In [ ]:
# Einfache Chunking-Funktion (Zeichen-basiert)

CHUNK_SIZE = 4000
CHUNK_OVERLAP = 200

joined = "\n".join(texts)

chunks = []
start = 0
while start < len(joined):
    end = start + CHUNK_SIZE
    chunk = joined[start:end]
    chunks.append(chunk)
    start = end - CHUNK_OVERLAP

len(chunks)


## 3) Embeddings erstellen und in Qdrant speichern
Wir erzeugen Embeddings über LiteLLM und speichern in Qdrant.


In [ ]:
# Embeddings erzeugen
# Hinweis: kann je nach Modell/Provider einige Zeit dauern.
embeddings = get_embeddings(chunks, llm_cfg, batch_size=512)
len(embeddings), len(embeddings[0])


In [ ]:
# Qdrant Collection anlegen und upsert
vector_size = len(embeddings[0])

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=qmodels.VectorParams(size=vector_size, distance=qmodels.Distance.COSINE),
)

points = []
for idx, (chunk, vector) in enumerate(zip(chunks, embeddings)):
    points.append(
        qmodels.PointStruct(
            id=idx,
            vector=vector,
            payload={"text": chunk},
        )
    )

# Batch-Upsert, um Payload-Limits zu vermeiden
BATCH_SIZE = 128
for start in range(0, len(points), BATCH_SIZE):
    batch = points[start : start + BATCH_SIZE]
    client.upsert(collection_name=collection_name, points=batch)


## 4) Schneller Test-Query
Kleine Retrieval-Abfrage als Smoke-Test.


In [ ]:
# Retrieval Test
query = "Wie gehe ich mit Sicherheitsmaßnahmen in der Organisation um?"
query_emb = get_embeddings([query], llm_cfg)[0]

response = client.query_points(
    collection_name=collection_name,
    query=query_emb,
    limit=5,
)
results = response.points

[res.payload.get("text", "")[:200] for res in results]


## 5) Leichter RAG-Chatbot
Wir holen Top-K Chunks aus Qdrant und schicken sie zusammen mit der Frage an GPT OSS 120B.


In [ ]:
# Leichter RAG-Chatbot (Context + LLM)
question = "Was sind empfohlene organisatorische Sicherheitsmaßnahmen?"

# Retrieval
query_emb = get_embeddings([question], llm_cfg)[0]
results = client.query_points(collection_name=collection_name, query=query_emb, limit=5).points
context = "".join([res.payload.get("text", "") for res in results])

messages = [
    {"role": "system", "content": "Du bist ein hilfreicher Assistent. Antworte kurz und cite den Kontext."},
    {"role": "user", "content": f"Frage: {question}\n\nKontext:\n{context}"},
]

response = chat_completion(messages, llm_cfg)
response


## 7) Evaluation mit RAGAS (CSV)
Wir nutzen die Datei `GrundschutzKI_Fragen-Antworten-Fundstellen.csv` als Test-Set.
Die Evaluation erstellt RAG-Antworten und vergleicht sie mit den erwarteten Antworten.


In [58]:
# RAGAS: Build evaluation records + answers (from CSV + Qdrant)
import pandas as pd
from pathlib import Path
from typing import List
from datasets import Dataset

def _retrieve_contexts(question: str, k: int, client, collection_name: str, llm_cfg) -> List[str]:
    query_emb = get_embeddings([question], llm_cfg, batch_size=1)[0]
    results = client.query_points(
        collection_name=collection_name,
        query=query_emb,
        limit=k,
    ).points
    return [res.payload.get("text", "") for res in results]

def _build_messages(question: str, contexts: list[str], fewshot: list[dict]) -> list[dict]:
    context_text = "\n\n".join(contexts)
    messages = [
        {
            "role": "system",
            "content": "Beantworte die Frage kurz, präzise und nutze ausschließlich den gelieferten Kontext! Antworte auf Deutsch. Die Antwort sollte maximal 2 Sätze lang sein.",
        },
    ]

    for ex in fewshot:
        messages.append(
            {
                "role": "user",
                "content": f"Frage: {ex['question']}\n\nKontext:\n<BEISPIEL-KONTEXT>",
            }
        )
        messages.append({"role": "assistant", "content": ex["answer"]})

    messages.append(
        {
            "role": "user",
            "content": f"Frage: {question}\n\nKontext:\n{context_text}",
        }
    )
    return messages

def _batch_generate_answers(messages_list: list[list[dict]], llm_cfg, batch_size: int, concurrency: int) -> list[str]:
    def _extract_content(resp) -> str:
        if isinstance(resp, dict):
            return resp["choices"][0]["message"]["content"]
        if hasattr(resp, "choices"):
            return resp.choices[0].message.content
        raise TypeError(f"Unexpected response type: {type(resp)}")
    import litellm

    answers: list[str] = []
    try:
        if hasattr(litellm, "batch_completion"):
            for start in range(0, len(messages_list), batch_size):
                batch = messages_list[start : start + batch_size]
                resp = litellm.batch_completion(
                    model=llm_cfg.model,
                    messages=batch,
                    api_key=llm_cfg.api_key,
                    api_base=llm_cfg.api_base,
                )
                if isinstance(resp, list):
                    batch_answers = [_extract_content(r) for r in resp]
                else:
                    batch_answers = [r["message"]["content"] for r in resp.get("data", [])]
                answers.extend(batch_answers)
            return answers
        raise AttributeError
    except Exception:
        async def _aget_one(msgs):
            return await litellm.acompletion(
                model=llm_cfg.model,
                messages=msgs,
                api_key=llm_cfg.api_key,
                api_base=llm_cfg.api_base,
            )
def build_eval_records(
    csv_path: str = "../data/data_evaluation/GSKI_Fragen-Antworten-Fundstellen.csv",
    sample_size: int = 10,
    top_k: int = 5,
    batch_size: int = 16,
    concurrency: int = 8,
) -> list[dict]:
    """Create RAGAS records with answers using LiteLLM + Qdrant."""
    llm_cfg = load_llm_config()
    vec_cfg = load_vectordb_config()
    qdrant_client = get_qdrant_client(vec_cfg)
    collection_name = vec_cfg.collection or "grundschutz_xml"

    df = pd.read_csv(Path(csv_path), sep=";", encoding="utf-8-sig")

    # Few-shot from first row
    first_row = df.iloc[0]
    fewshot = [{"question": first_row["Frage"], "answer": first_row["Antwort"]}]

    records = []
    questions = []
    contexts_list = []

    for _, row in df.iloc[1 : 1 + sample_size].iterrows():
        question = row["Frage"]
        ground_truth_answer = row["Antwort"]
        ground_truth_context = row["Fundstellen im IT-Grundschutz-Kompendium 2023"]
        contexts = _retrieve_contexts(question, top_k, qdrant_client, collection_name, llm_cfg)

        questions.append(question)
        contexts_list.append(contexts)
        records.append({"question": question, "answer": "", "contexts": contexts, "ground_truth_answer": ground_truth_answer, "ground_truth_context": ground_truth_context})

    messages_list = [_build_messages(q, ctx, fewshot) for q, ctx in zip(questions, contexts_list)]
    answers = _batch_generate_answers(messages_list, llm_cfg, batch_size, concurrency)

    for rec, ans in zip(records, answers):
        rec["answer"] = ans

    return Dataset.from_list(records)


In [57]:
dataset = build_eval_records(sample_size=50, top_k=5, batch_size=16, concurrency=10)
dataset

FileNotFoundError: [Errno 2] No such file or directory: '../GrundschutzKI_Fragen-Antworten-Fundstellen.csv'

In [25]:
print("QUESTION:", dataset[0]['question'])
print()
print("ANSWER:", dataset[0]['answer'])
print()
print("GROUND TRUTH:", dataset[0]['ground_truth'])

QUESTION: Welche grundlegenden Sicherheitsmaßnahmen müssen beim Einrichten eines Webservers ergriffen werden?

ANSWER: Beim Einrichten eines Webservers muss zunächst eine sichere Grundkonfiguration vorgenommen werden – der Webserver‑Prozess läuft unter einem Konto mit minimalen Rechten in einer gekapselten Umgebung, nicht benötigte Module und Schreibrechte werden deaktiviert, das WWW‑Wurzelverzeichnis wird strikt eingegrenzt und alle Dateien sowie Konfigurationen vor unbefugtem Zugriff geschützt. Zusätzlich sind restriktive HTTP‑Header (z. B. Strict‑Transport‑Security, Content‑Type, X‑Content‑Type‑Options, Cache‑Control), sichere Cookie‑Attribute (secure, SameSite, httponly), der Einsatz einer Web‑Application‑Firewall, umfassende Protokollierung, kryptografisch gesicherte Passwortspeicherung, Prüfung von Datei‑Uploads sowie regelmäßige Penetrationstests und Revisionen erforderlich.

GROUND TRUTH: Nach der Installation eines Webservers muss eine sichere Grundkonfiguration vorgenommen we

## 8) Interpretation der RAGAS-Metriken
**answer_relevancy**: Wie gut die Antwort die Frage adressiert (höher = besser).
**faithfulness**: Wie gut die Antwort durch den gegebenen Kontext gedeckt ist (höher = weniger Halluzination).
**context_precision**: Wie viel des gelieferten Kontextes wirklich relevant ist (höher = weniger Rauschen).
**context_recall**: Wie viel der relevanten Informationen im Kontext enthalten ist (höher = besseres Retrieval).

**Daumenregel:**
- Hoher Recall + niedrige Relevanz ⇒ zu viel Kontext oder schlechte Antwortformulierung.
- Hohe Precision + niedrige Faithfulness ⇒ Antwort nutzt Kontext nicht sauber.
- Niedrige Precision + hohe Recall ⇒ Retrieval liefert viel, aber unpräzise.


In [26]:
llm_cfg

LLMConfig(api_base='http://10.127.129.0:4000/v1/', api_key='sk-kW0pG01NT9iGe4OhNlYFyw', model='openai/gpt-oss-120b', embedding_model='openai/octen-embedding-8b')

In [31]:
# RAGAS metrics with LiteLLM proxy config from .env
from ragas.llms import llm_factory
from ragas.embeddings.litellm_provider import LiteLLMEmbeddings
from ragas.metrics.collections import ContextPrecision, ContextRecall, Faithfulness, AnswerCorrectness
import instructor
import litellm
from litellm_client import load_llm_config
from ragas.embeddings.litellm_provider import LiteLLMEmbeddings


llm_cfg = load_llm_config()
litellm.api_base = llm_cfg.api_base
litellm.api_key = llm_cfg.api_key

# Uses the model + api_base + api_key from .env (LiteLLM proxy)
# from openai import AsyncOpenAI
# client = AsyncOpenAI(api_key=llm_cfg.api_key, base_url=llm_cfg.api_base)
client = instructor.from_litellm(litellm.acompletion, mode=instructor.Mode.MD_JSON)
llm = llm_factory(
    llm_cfg.model,
    client=client,
    adapter="litellm",
    model_args={"temperature": 0.2},
)
embeddings = LiteLLMEmbeddings(
    model=llm_cfg.embedding_model,
    api_key=llm_cfg.api_key,
    api_base=llm_cfg.api_base,
    encoding_format="float",
)



In [32]:
from ragas.metrics.collections import (
    ContextPrecision,
    ContextRecall,
    Faithfulness,
    AnswerCorrectness,
)
import asyncio

scorers = {
    "context_precision": ContextPrecision(llm=llm),
    "context_recall": ContextRecall(llm=llm),
    "faithfulness": Faithfulness(llm=llm),
    "answer_correctness": AnswerCorrectness(llm=llm, embeddings=embeddings),
}

async def _score_row(row, sem):
    async with sem:
        return {
            "context_precision": (await scorers["context_precision"].ascore(
                user_input=row["question"],
                reference=row["ground_truth_context"],
                retrieved_contexts=row["contexts"],
            )).value,
            "context_recall": (await scorers["context_recall"].ascore(
                user_input=row["question"],
                reference=row["ground_truth_context"],
                retrieved_contexts=row["contexts"],
            )).value,
            "faithfulness": (await scorers["faithfulness"].ascore(
                user_input=row["question"],
                response=row["answer"],
                retrieved_contexts=row["contexts"],
            )).value,
            "answer_correctness": (await scorers["answer_correctness"].ascore(
                user_input=row["question"],
                response=row["answer"],
                reference=row["ground_truth_answer"],
            )).value,
        }

async def score_dataset_batched(ds, batch_size=10, concurrency=5):
    sem = asyncio.Semaphore(concurrency)
    rows = list(ds)
    results = []

    for i in range(0, len(rows), batch_size):
        batch = rows[i : i + batch_size]
        tasks = [asyncio.create_task(_score_row(r, sem)) for r in batch]
        results.extend(await asyncio.gather(*tasks))

    return results

scores = await score_dataset_batched(dataset, batch_size=20, concurrency=20)

# scores is a list of dicts, one per row
stats = {
    k: {
        "avg": sum(s[k] for s in scores) / len(scores),
        "min": min(s[k] for s in scores),
        "max": max(s[k] for s in scores),
    }
    for k in scores[0].keys()
}
print(stats)




{'context_precision': {'avg': 0.9209010839804745, 'min': 0.249999999975, 'max': 0.99999999998}, 'context_recall': {'avg': 0.9735772357723577, 'min': 0.5, 'max': 1.0}, 'faithfulness': {'avg': 0.722765107521205, 'min': 0.0, 'max': 1.0}, 'answer_correctness': {'avg': 0.6200686538614504, 'min': 0.13106508565833477, 'max': 0.9853846488050791}}


In [33]:
from IPython.display import display, Markdown
display(Markdown(
f"""**RAG Evaluation Summary (Percentages)**

- **Context precision**: {stats['context_precision']['avg']*100:.1f}%  
  (min {stats['context_precision']['min']*100:.1f}%, max {stats['context_precision']['max']*100:.1f}%)
- **Context recall**: {stats['context_recall']['avg']*100:.1f}%  
  (min {stats['context_recall']['min']*100:.1f}%, max {stats['context_recall']['max']*100:.1f}%)
- **Faithfulness**: {stats['faithfulness']['avg']*100:.1f}%  
  (min {stats['faithfulness']['min']*100:.1f}%, max {stats['faithfulness']['max']*100:.1f}%)
- **Answer correctness**: {stats['answer_correctness']['avg']*100:.1f}%  
  (min {stats['answer_correctness']['min']*100:.1f}%, max {stats['answer_correctness']['max']*100:.1f}%)
"""
))


**RAG Evaluation Summary (Percentages)**

- **Context precision**: 92.1%  
  (min 25.0%, max 100.0%)
- **Context recall**: 97.4%  
  (min 50.0%, max 100.0%)
- **Faithfulness**: 72.3%  
  (min 0.0%, max 100.0%)
- **Answer correctness**: 62.0%  
  (min 13.1%, max 98.5%)


In [34]:
def daumenregel_hinweise(
    precision: float,
    recall: float,
    faithfulness: float,
    *,
    high: float = 0.75,
    low: float = 0.5,
):
    hints = []

    if recall >= high and precision <= low:
        hints.append("Hoher Recall + niedrige Relevanz ⇒ zu viel Kontext oder schlechte Antwortformulierung.")

    if precision >= high and faithfulness <= low:
        hints.append("Hohe Precision + niedrige Faithfulness ⇒ Antwort nutzt Kontext nicht sauber.")

    if precision <= low and recall >= high:
        hints.append("Niedrige Precision + hohe Recall ⇒ Retrieval liefert viel, aber unpräzise.")

    if not hints:
        hints.append("Keine auffällige Daumenregel‑Kombination erkannt.")

    return hints

avg = stats
hints = daumenregel_hinweise(
    precision=avg["context_precision"]["avg"],
    recall=avg["context_recall"]["avg"],
    faithfulness=avg["faithfulness"]["avg"],
)

for h in hints:
    print("-", h)

- Keine auffällige Daumenregel‑Kombination erkannt.


# 8) DSPy
DSPy ist ein Framework, das LLM‑Prompts und Programme systematisch optimiert, statt sie nur manuell zu schreiben.
Wir nutzen DSPy, um Antworten konsistenter, faktengetreuer und besser an unsere Aufgaben anzupassen.
Gerade bei RAG hilft DSPy, die Nutzung des Kontextes zu verbessern und die Qualität der Antworten messbar zu steigern.


In [36]:
import dspy
# DSPy LLM (OpenAI-compatible via LiteLLM proxy)
dspy_llm = dspy.LM(
    model=llm_cfg.model,
    api_base=llm_cfg.api_base,
    api_key=llm_cfg.api_key,
    temperature=0.2,  # as recommended for benchmarking
)

dspy.configure(lm=dspy_llm)


In [ ]:

class RAGAnswer(dspy.Signature):
    """Answer the question using the provided context only."""
    question: str = dspy.InputField()
    context: str = dspy.InputField()
    answer: str = dspy.OutputField(desc="Antworte auf Deutsch, kurz und präzise, maximal 2–3 Sätze.")

class RAGModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(RAGAnswer)

    def forward(self, question, context):
        return self.predict(question=question, context=context)

rag = RAGModule()


In [39]:
for i in range(3):
    row = dataset[i]
    context = "\n\n".join(row["contexts"])
    pred = rag(question=row["question"], context=context)
    print(f"\n--- SAMPLE {i} ---")
    print("QUESTION:", row["question"])
    print("ANSWER:", pred.answer)
    print("GROUND TRUTH:", row["ground_truth"])



--- SAMPLE 0 ---
QUESTION: Welche grundlegenden Sicherheitsmaßnahmen müssen beim Einrichten eines Webservers ergriffen werden?
ANSWER: Beim Einrichten eines Webservers sind eine sichere Grundkonfiguration (Konto mit minimalen Rechten, Kapselung/Isolation, Deaktivierung nicht benötigter Module), restriktive Dateiberechtigungen und ein geschütztes WWW‑Wurzelverzeichnis, die Durchsetzung von HTTPS mit HSTS sowie sicheren HTTP‑Headern (Content‑Type, X‑Content‑Type‑Options, Cache‑Control) und sicheren Cookie‑Attributen (Secure, SameSite, HttpOnly) erforderlich. Zusätzlich müssen umfassende Protokollierung, regelmäßige Penetrationstests/Revisionen, Malware‑Scans für Uploads und, bei erhöhtem Schutzbedarf, eine Web‑Application‑Firewall eingesetzt werden.
GROUND TRUTH: Nach der Installation eines Webservers muss eine sichere Grundkonfiguration vorgenommen werden. Dazu gehört die Zuweisung des Webserver-Prozesses einem Konto mit minimalen Rechten, die Ausführung in einer gekapselten Umgebung (

In [ ]:
import numpy as np

def cosine_sim(a, b):
    a = np.asarray(a); b = np.asarray(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# dspy_answers: Liste der DSPy-Outputs in gleicher Reihenfolge wie dataset
# dataset["answer"]: bisherige Antworten

async def cosine_similarity_dspy_vs_baseline(dspy_answers, baseline_answers, embeddings):
    sims = []
    for dspy_ans, base_ans in zip(dspy_answers, baseline_answers):
        emb1 = await embeddings.aembed_text(dspy_ans)
        emb2 = await embeddings.aembed_text(base_ans)
        sims.append(cosine_sim(emb1, emb2))
    return sims
dspy_answers = []
for i in range(len(dataset)):
    row = dataset[i]
    context = "\n\n".join(row["contexts"])
    pred = rag(question=row["question"], context=context)
    dspy_answers.append(pred.answer)

sims = await cosine_similarity_dspy_vs_baseline(
    dspy_answers,
    dataset["answer"],
    embeddings,
)

print("avg cosine:", sum(sims) / len(sims))
print("min:", min(sims), "max:", max(sims))


avg cosine: 0.9184638717993329
min: 0.7410496258342404 max: 0.9836617026475417
